## Learning pyspark
Just basic function of pyspark to show how to utilize this data processing method  
I also integrated with git functions for version control

In [4]:
!pip install pyspark

     -------------------------------------- 317.3/317.3 MB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 200.5/200.5 kB 393.1 kB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849816 sha256=cafaf667e5a3fb0e96fb08c750b336c36800d3bd4389b7a381c36af255762639
  Stored in directory: c:\users\khang_\appdata\local\pip\cache\wheels\1a\ce\a4\94a21408027efb9651508706a4172b868f507f51a938c4636a
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9
  Using cached pyspark-3.5.4-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Can't uninstall 'py4j'. No files were found to uninstall.


In [3]:
import pyspark

ModuleNotFoundError: No module named 'pyspark'